# The Big Short

---

### Import Libraries and Dependencies

In [5]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

### Read CSV into Pandas DataFrame

In [6]:
# Set the file path
filepath = Path("../Resources/vnq.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index
vnq_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
vnq_df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/07,77.339996,77.709999,76.400002,44.707417,525700
1,1/4/07,77.230003,77.260002,76.629997,44.765377,166100
2,1/5/07,77.099998,77.099998,75.750000,43.936401,143700
3,1/8/07,76.000000,76.000000,75.430000,44.000164,87000
4,1/9/07,75.730003,76.959999,75.730003,44.556675,136300


### Generate a Dual Moving Average Crossover Trading Signal to _short_ the market

In [7]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = vnq_df.loc[:, ['Date', 'Close']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Close'].rolling(window=long_window).mean()

# Initialize the new `Signal` column
signals_df['Signal'] = 0.0

# Generate the trading signal (1 or 0) to when the short window is less than the long
# Note: Use 1 when the SMA50 is less than SMA100 and 0 for when it is not.
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] < signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
12/17/09,29.639492,28.167839,27.217638,0.0,0.0
12/18/09,29.835062,28.211270,27.294345,0.0,0.0
12/21/09,30.239697,28.258072,27.367152,0.0,0.0
12/22/09,30.556675,28.312024,27.443329,0.0,0.0
12/23/09,30.920845,28.381082,27.519007,0.0,0.0
12/24/09,31.367579,28.438977,27.586397,0.0,0.0
12/28/09,31.517859,28.504734,27.645874,0.0,0.0
12/29/09,31.046522,28.575901,27.702575,0.0,0.0
12/30/09,31.094339,28.636155,27.745596,0.0,0.0


In [10]:
signals_df[signals_df['Entry/Exit']!=0] 

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
1/3/07,44.707417,NaN,NaN,0.0,NaN
5/25/07,43.461689,45.999724,46.646618,1.0,1.0
10/23/07,42.738773,41.916470,41.896646,0.0,-1.0
12/17/07,36.302608,40.653431,40.758031,1.0,1.0
4/10/08,39.311516,37.223229,37.195207,0.0,-1.0
7/16/08,35.972519,38.544135,38.593600,1.0,1.0
5/21/09,20.512068,18.755658,18.745874,0.0,-1.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [4]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)